## AI Foundry - Agents v2

For the time being, install Preview version of Foundry SDK to use Agents v2 API:

``` bash
pip install azure-ai-projects --pre
```

### Environment Setup

In [1]:
# Import required packages
import os
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from azure.identity import DefaultAzureCredential

In [2]:
# Set environment variables
PROJECT_ENDPOINT = os.environ.get("AZURE_FOUNDRY_PROJECT_ENDPOINT")
MODEL_DEPLOYMENT = os.environ.get("AZURE_FOUNDRY_GPT_MODEL")
AGENT_NAME = "Demo-PromptAgent-v2"

### Agent Creation

In [3]:
# Define AI Project Client
ai_client = AIProjectClient(
    endpoint = PROJECT_ENDPOINT,
    credential = DefaultAzureCredential()
)

In [4]:
# Create AI Foundry prompt agent v2
ai_agent = ai_client.agents.create_version(
    agent_name = AGENT_NAME,
    definition = PromptAgentDefinition(
        model = MODEL_DEPLOYMENT,
        instructions = "You are a helpful assistant that answers general questions",
    ),
)

print(f"Agent created (\n  - id: {ai_agent.id},\n  - name: {ai_agent.name},\n  - version: {ai_agent.version}\n)")

Agent created (
  - id: Demo-PromptAgent-v2:1,
  - name: Demo-PromptAgent-v2,
  - version: 1
)


### Chat with Agent

In [5]:
# Define Azure OpenAI Client
openai_client = ai_client.get_openai_client()

In [6]:
# Create multi-turn conversation
conversation = openai_client.conversations.create()

print(f"Created conversation (id: {conversation.id})")

Created conversation (id: conv_01d2bc48c334471600YkC1Sp63dWJWAiTZoWxJRjINEeYU7of3)


In [7]:
# Define helper function
def agent_chat(prompt):
    response = openai_client.responses.create(
        conversation = conversation.id,
        extra_body = {
            "agent": {
                "name": AGENT_NAME,
                "type": "agent_reference"
            }
        },
        input = prompt,
    )

    return response.output_text

In [8]:
# Query #1
prompt1 = "What is the capital of Ecuador?"

print(f"User: {prompt1}\n")
print(f"Agent: {agent_chat(prompt1)}")

User: What is the capital of Ecuador?

Agent: The capital of Ecuador is Quito.


In [9]:
# Query #2
prompt2 = "And what is the population there?"

print(f"User: {prompt2}\n")
print(f"Agent: {agent_chat(prompt2)}")

User: And what is the population there?

Agent: As of recent estimates, the population of Quito, Ecuador, is approximately 2.8 million people. However, population figures can vary depending on the source and the year of the estimate.


### Housekeeping

In [10]:
# Delete agent version
ai_client.agents.delete_version(
    agent_name = AGENT_NAME,
    agent_version = ai_agent.version
)

print(f"Deleted agent: {AGENT_NAME}, version: {ai_agent.version}")

Deleted agent: Demo-PromptAgent-v2, version: 1
